In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture --no-stderr
!pip install python-dotenv langchain-community langchain-core langchain langchain-openai langchain-chroma

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [ ]:
# .env 파일에서 환경 변수 로드 (.env 파일에는 OPENAI API 키값을 적으면 됩니다. -> OPENAI_API_KEY=...)
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

# **이전 대화를 포함한 메시지 전달**

In [ ]:
# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate
from langchain_opneai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")

# 프롬프트 템플릿 정의: 금융 상담 역할
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다."),
        # 대화 이력 추가
        ("placeholder", "{messages}"),
    ]
)

In [ ]:
# 프롬프트와 모델을 연결하여 체인 생성
chain = prompt | chat

In [ ]:
# 이전 대화를 포함한 메시지 전달
ai_msg = chain.invoke(
    {
        "messages": [
            # 사용자의 첫 질문
            ("human", "저축을 늘리기 위해 무엇을 할 수 있나요?"),
            # 챗봇의 답변
            ("ai", "저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.")
            # 사용자의 재확인 질문
            ("human", "방금 뭐라고 했나요?")
        ],
    }
)

# 챗봇의 응답 출력
print(ai_msg.content)

# **ChatMessageHistory를 사용한 메시지 관리**

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory

# 대화 이력 저장을 위한 클래스 초기화
chat_history = ChatMessageHistory()

# 사용자 메시지 추가
chat_history.add_user_message("저축을 늘리기 위해 무엇을 할 수 있나요?")
chat_history.add_ai_message("저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.")

In [ ]:
# 새로운 질문 추가 후 다시 체인 실행
chat_history.add_user_message("방금 뭐라고 했나요?")
ai_response = chain.invoke({"messages": chat_history.messages})

# 챗봇은 이전 메시지를 기억하여 답변한다
print(ai_reponse.content)

# **RunnableWithMessageHistory를 사용한 메시지 관리**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# 시스템 메시지와 대화 이력을 사용하는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오."),
        # 이전 대화 이력
        ("placeholder", "{chat_history}"),
        # 사용자의 새로운 질문
        ("human", "{input}"),
    ]
)

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
# 대화 이력을 관리할 체인 설정

# 대화에서 오간 메시지(Human / AI / System / Tool 등)를 순서대로 저장/조회하는 역할
# 대화의 "메모리" 역할을 하는 기본 구현체
# "대화 이력 저장소 하나 만든다" 라는 의미
chat_history = ChatMessageHistory()
chain = prompt | chat

In [ ]:
# RunnableWithMessageHistory 클래스를 사용해 체인을 감싼다
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    # 세션 ID에 따라 대화 이력을 불러오는 함수
    # 세션별로 대화 이력을 관리하는 함수
    # 여기서는 간단히 chat_history라는 변수를 항상 반환하도록 함
    # 실제로는 세션 ID 별로 Redis, DB, 메모리 등에서 꺼내올 수 있음

    # RunnableWithMessageHistory는 세션별로 어떤 대화 이력(ChatMessageHistory)을 쓸지 알아야 한다
    # 이건 간단한 예시라서 세션 ID를 무시하고, 항상 같은 chat_history 객체를 반환
    lambda session_id: chat_history,
    # 입력 메시지의 키 설정
    # 사용자의 입력 텍스트가 딕셔너리에서 어떤 키로 들어올지 지정
    input_messages_key="input",
    # 대화 이력의 키 설정
    # 체인 내부에서 대화 이력을 어떤 이름으로 받을지 지정
    # PromptTemplate 안에서는 {chat_history} 같은 식으로 참조 가능
    history_messages_key="chat_history",
)

In [ ]:
# 질문 메시지 체인 실행
chain_with_message_history.invoke(
    {"input": "저축을 늘리기 위해 무엇을 할 수 있나요?"},
    # {"configurable": {...}} 는 LangChain에서 실행 옵션(메타데이터)을 넣는 자리
    # 그중 session_id는 RunnableWithMessageHistory가 어떤 세션의 대화 이력을 불러와야 하는지 결정할 때 사용.
    # 지금은 "unused"라는 문자열을 넣었는데, 위에서 만든 람다가 session_id를 무시하기 때문에 사실 아무 의미가 없다.
    {"configurable": {"session_id": "unused"}},
).content

In [ ]:
# 새로운 입력 메시지를 추가하고 체인 실행
chain_with_message_history.invoke(
    {"input": "내가 방금 뭐라고 했나요?"},
    {"configurable": {"session_id": "unused"}}
).content